In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
data=pd.read_csv("../../data/006/spam.csv",encoding='latin-1')

In [3]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [4]:
data1=data.iloc[:,0:2]

In [5]:
data1.head(5)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data1.isnull().sum().sum()

0

In [7]:
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder()

data['v1'] = encoder.fit_transform(data['v1'])
class_mappings = {index: label for index, label in enumerate(encoder.classes_)}

In [8]:
class_mappings

{0: 'ham', 1: 'spam'}

In [9]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
data1=data[['v2', 'v1']]
data1.columns=['text','labels']
data1.head()

,text,labels
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [11]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from sklearn.model_selection import train_test_split

text_train,text_test,label_train,label_test=train_test_split(data1['text'],data1['labels'] ,test_size=0.2, random_state=42)

In [19]:
class SpamDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx],dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [20]:
train_dataset = SpamDataset(text_train, label_train)
test_dataset = SpamDataset(text_train,label_train)

In [22]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    eval_strategy="steps",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [23]:
trainer.train()

  0%|          | 0/837 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
# # Install dependencies if you haven't already
# # !pip install transformers
# # !pip install torch
# # !pip install sklearn

# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# from transformers import Trainer, TrainingArguments
# import torch
# from torch.utils.data import Dataset

# # Step 1: Load the dataset
# # Assuming you've already loaded the SMS Spam Collection dataset
# df = pd.read_csv('spam.csv')  # or wherever your dataset is stored
# df = df[['text', 'target']]  # Select relevant columns ('text' and 'target')

# # Step 2: Encode labels (spam = 1, ham = 0)
# label_encoder = LabelEncoder()
# df['target'] = label_encoder.fit_transform(df['target'])  # spam=1, ham=0

# # Step 3: Split into train and test sets
# train_texts, test_texts, train_labels, test_labels = train_test_split(
#     df['text'], df['target'], test_size=0.2, random_state=42)

# # Step 4: Load tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# # Step 5: Tokenize the text
# class SpamDataset(Dataset):
#     def __init__(self, texts, labels):
#         self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512)
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels.iloc[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# train_dataset = SpamDataset(train_texts, train_labels)
# test_dataset = SpamDataset(test_texts, test_labels)

# # Step 6: Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size for training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
#     evaluation_strategy="steps",
# )

# # Step 7: Train the model using Hugging Face Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
# )

# trainer.train()

# # Step 8: Evaluate the model
# eval_results = trainer.evaluate()
# print(f"Evaluation Results: {eval_results}")

# # Step 9: Inference (predict on new data)
# def predict(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1)
#     return "spam" if predictions == 1 else "ham"

# # Example usage
# print(predict("Congratulations! You've won a $1000 Walmart gift card."))
# print(predict("Hello, let's meet up for lunch tomorrow."))